# Imports

In [ ]:
import math
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras
import sklearn.metrics as metrics

%matplotlib inline
np.random.seed(1)

In [ ]:
# temp_path = "/Users/pranavupadhyayula/PycharmProjects/LipReading/dataset"
# temp_path = "/Users/pranavupadhyayula/Downloads/cropped_data"
temp_path="/Users/sushil/OneDrive/Removable Disk/2Stanford/Spring/CS 221/cropped_data"

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Average-CNN Model

In [ ]:
X = []
y = []
# person = ["F01", "F02", "F04", "F05", "F06", "F07", "F08", "F11", "M01", "M02", "M04", "M07", "M08"]
person = ["F01"]
# prob = ["phrases", "words"]
prob = ["phrases"]
# label = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
label = ["01"]
instance = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
instance = ["01"]
min_sequence_length = 4

# AVERAGE IMAGES
for p1 in person:
    for p2 in prob:
        for l in label:           
            for i in instance:   
                X_train = np.zeros((250, 200, 3), dtype=np.float64)
                count = 0
                curr_path = temp_path + "/" + p1 + "/" + p2 + "/" + l + "/" + i
                print(curr_path)
                
                for filename in os.listdir(curr_path):
                    if filename.endswith(".jpg"): 
                        count += 1
                        image_path = os.path.join(curr_path, filename)
                        im = Image.open(image_path)
                        X_i = np.array(im.getdata(), dtype=np.float64)
                        X_i /= 255
                        X_i = X_i.reshape((250, 200, 3))       
                        X_train += X_i

                X_train /= count
                X.append(X_train)
                y.append(int(l) - 1)

                
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

# R-CNN Model

In [ ]:
# SEQUENCE IMAGES
for p1 in person:
    for p2 in prob:
        for l in label:           
            for i in instance:   
                curr_path = temp_path + "/" + p1 + "/" + p2 + "/" + l + "/" + i
                print(curr_path)
                
                num_img = 0 
                img_files = []
                for filename in os.listdir(curr_path):
                    if filename.endswith(".jpg"): 
                        image_path = os.path.join(curr_path, filename)
                        im = Image.open(image_path)
                        img_files.append(im)
                        num_img += 1
                
                start_index = (num_img // min_sequence_length) + ((num_img % 4) // 2) - 1 if num_img%2 == 0 else (num_img // min_sequence_length) + ((num_img % 4) // 2)
#                 start_index = 0
                skip_size = num_img // min_sequence_length
#                 skip_size = int(round(num_img * 1.0 / min_sequence_length))
                
                print(start_index, skip_size, num_img)
                
                count = 0
                X_train = []
                for i in range(start_index, num_img, skip_size):
                    if (len(X_train) < 4):
                        im = img_files[i]
                        X_i = np.array(im.getdata(), dtype=np.float64)
                        X_i = X_i.reshape((50, 50, 3))
                        X_train.append(X_i)
                
                print(len(X_train))
                X_train = np.array(X_train)
                X_train = X_train.reshape((min_sequence_length, 50, 50, 3))
                X.append(X_train)
                y.append(int(l) - 1)
                
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

# Model Training

In [ ]:
## CNN W AVERAGED IMAGES
# model = keras.Sequential()
# model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(640, 480, 3)))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
# model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

## CNN W IMAGE SEQUENCES
# model = keras.Sequential()
# model.add(keras.layers.Conv3D(64, kernel_size=2, activation='relu', input_shape=(min_sequence_length, 50, 50, 3), data_format="channels_last"))
# # model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=None, padding='valid', data_format=None))
# # model.add(keras.layers.Conv3D(32, kernel_size=2, activation='relu', data_format="channels_last"))
# # model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=None, padding='valid', data_format=None))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

## RNN W IMAGE SEQUENCES
# model = keras.Sequential()
# model.add(keras.layers.Embedding(input_dim=1300, input_length = 4, output_dim=10))
# model.add(keras.layers.LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

## LSTM CNN W IMAGE SEQUENCES
# model = keras.Sequential()
# model.add(keras.layers.TimeDistributed(keras.layers.Conv3D(64, kernel_size=2, activation='relu', input_shape=(min_sequence_length, 50, 50, 3), data_format="channels_last")))
# model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=None, padding='valid', data_format=None)))
# model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
# model.add(keras.layers.LSTM(10))
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

# model = keras.Sequential()
# model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(min_sequence_length, 50, 50, 3))))
# model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)))
# model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4, kernel_size=3, activation='relu')))
# model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)))
# model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
# model.add(keras.layers.LSTM(10))
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

# model = keras.Sequential()
# model.add(keras.layers.ConvLSTM2D(16, kernel_size=3, activation='relu', input_shape=(min_sequence_length, 50, 50, 3), data_format='channels_last'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
# model.add(keras.layers.ConvLSTM2D(4, kernel_size=3, activation='relu', data_format='channels_last'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.LSTM(10))
# model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.9, test_size=0.1, shuffle= True)
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))
# model.fit(X, y, epochs=50, validation_data=(X, y))

# model.fit(X, y, epochs=2, validation_split=0.2)





# Model Testing

In [ ]:
model = keras.Sequential()
model.add(keras.applications.vgg16.VGG16(classes=10, weights=None))


model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.9, test_size=0.1, shuffle= True)
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))










### Script to plot the loss

In [ ]:
from matplotlib import pyplot as plt
from IPython.display import clear_output
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

# LSTM Model Testing 

In [ ]:
import pickle

X = pickle.load( open( "data_X.p", "rb" ) )
y = pickle.load( open( "data_Y.p", "rb" ) )

X = np.array(X)
y = np.array(y)

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pickle

np.random.seed(7)

X = sequence.pad_sequences(X, maxlen=27)

max_x_val = max(X.flatten()) + 1

embedding_vecor_length = 4
model = Sequential()
model.add(Embedding(max_x_val, embedding_vecor_length, input_length=27))
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(10))
model.add(Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
print(model.summary())

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.9, test_size=0.1, shuffle= True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), batch_size=50)






# Add Data Augmentation

In [ ]:
X = []
y = []
person = ["F01", "F02", "F04", "F05", "F06", "F07", "F08", "F11", "M01", "M02", "M04", "M07", "M08"]
# person = ["F01"]
# prob = ["phrases", "words"]
prob = ["phrases"]
label = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
# label = ["01"]
instance = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
# instance = ["01"]
min_sequence_length = 4

for p1 in person:
    for p2 in prob:
        for l in label:           
            for i in instance:   
                X_train = np.zeros((250, 200, 3), dtype=np.float64)
                curr_path = temp_path + "/" + p1 + "/" + p2 + "/" + l + "/" + i
                for filename in os.listdir(curr_path):
                    if filename.endswith(".jpg"): 
                        image_path = os.path.join(curr_path, filename)
                        im = Image.open(image_path)
                        X_i = np.array(im.getdata(), dtype=np.float64)
                        X_i /= 255
                        X_i = X_i.reshape((224, 224, 3))       
                        X_train += X_i

                X_train /= count
                X.append(X_train)
                y.append(int(l) - 1)

# CAMS

In [ ]:
import math
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from vis.visualization import visualize_cam, visualize_saliency, overlay
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras
import matplotlib.cm as cm
from vis.utils import utils
from keras import activations


#gets desired layer
last_layer = utils.find_layer_idx(model, "lstm_1")

#loads images to get CAMs of
img1 = utils.load_img(temp_path+'/F07/phrases/01/01/color_007.jpg',target_size=(250, 200, 3))
img2 = utils.load_img(temp_path+'/F09/phrases/07/07/color_007.jpg', target_size=(250, 200, 3))
print(utils.get_img_shape(img1))
f, ax = plt.subplots(1, 2)
ax[0].imshow(img1)
ax[1].imshow(img2)
#displays original images
plt.show()

for modifier in [None]:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img1, img2]):    
        grads = visualize_cam(model, layer_idx=last_layer, filter_indices=[4], 
                              seed_input=img)         
        #Overlays the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        plt.imshow(overlay(grads, img))
        plt.show()

# Confusion Matrix using Seaborn

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = [[13,0,1,1,0,4,0,3,0,4], 
        [0,11,7,0,1,1,2,3,0,1], 
        [1,4,18,0,0,1,1,0,0,1], 
        [0,0,0,15,4,0,3,0,4,0], 
        [1,0,0,5,10,0,6,0,4,0], 
        [4,0,0,2,0,9,0,5,0,6], 
        [2,1,1,5,2,1,10,2,1,1],
        [2,0,2,0,0,3,0,16,0,3], 
        [1,2,1,2,1,1,3,0,14,1],
        [1,0,2,1,0,4,0,3,0,15]]
columns = ['Stop navigation.',
          'Excuse me.',
          'I am sorry.',
          'Thank you.',
          'Good bye.',
          'I love this game.',
          'Nice to meet you.',
          'You are welcome.',
          'How are you?',
          'Have a good time.']
df_cm = pd.DataFrame(array, index = columns,
                  columns = columns)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="Reds")